# Example 1: Get protein information, run BLAST, and dump the results into a database


In [1]:
%reload_ext autoreload
%autoreload 2
from pyEED.core import ProteinInfo

## Query NCBI

The pyEED library is centered around the `ProteinSequence` object, which integrates available information on protein sequence, corresponding nucleotide sequence, as well as regions and sites within the sequences. The `ProteinSequence` can be initialized directly with a protein sequence accession number.

In [8]:
aldolase = ProteinInfo.from_ncbi("NP_001287541.1")
print(aldolase)

ProteinInfo
├── id = proteininfo53
├── source_id = NP_001287541.1
├── name = aldolase 1, isoform M
├── sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = organism53
│       ├── name = Drosophila melanogaster
│       ├── taxonomy_id = taxon:7227
│       ├── domain = Eukaryota
│       ├── kingdom = Metazoa
│       ├── phylum = Arthropoda
│       ├── tax_class = Insecta
│       ├── order = Diptera
│       ├── family = Drosophilidae
│       ├── genus = Drosophila
│       └── species = melanogaster
├── regions
│   └── 0
│       └── ProteinRegion
│           ├── id = proteinregion51
│           ├── name = Glycolytic
│           ├

## BLAST search

In [3]:
blast_results = aldolase.pblast(n_hits=50, e_value=1e-50)

🏃🏼‍♀️ Running PBLAST
├── protein name: intracellular PHB depolymerase
├── organism: Cupriavidus necator H16
├── e-value: 1e-50
└── max hits: 50


Fetching protein sequences: 50it [00:01, 25.73it/s]


In [5]:
print(blast_results[1])

ProteinInfo
├── id = proteininfo2
├── source_id = WP_010809034.1
├── name = polyhydroxyalkanoate depolymerase
├── sequence = MLYQLHEFQRSILHPLTAWAQATAKTFTNPLSPLSLVPGAPRLAAGYELLYRLGKEYEKPAFDIKSVRSNGRDIPIVEQTVLEKPFCKLVRFKRYADDPETIKLLKDEPVVLVAAPLSGHHATLLRDTVRTLLQDHKVYVTDWIDARMVPVEEGAFHLSDYIYYIQEFIRHIGAENLHVISVCQPTVPVLAAISLMASAGEKTPRTMTMMGGPIDARKSPTAVNSLATNKSFEWFENNVIYTVPANYPGHGRRVYPGFLQHAGFVAMNPDRHLSSHYDFYLSLVEGDADDAEAHVRFYDEYNAVLDMAAEYYLDTIREVFQEFRLANGTWAIDGNPVRPQDIKSTALMTVEGELDDISGAGQTAAAHDLCAGIPKTRKQHLNAAHCGHYGIFSGRRWREEIYPQLRDFIRKYHQASATR
├── organism
│   └── Organism
│       ├── id = organism2
│       ├── name = Cupriavidus
│       ├── taxonomy_id = taxon:106589
│       ├── domain = Bacteria
│       ├── kingdom = Pseudomonadota
│       └── phylum = Burkholderiales
├── regions
│   └── 0
│       └── ProteinRegion
│           ├── id = proteinregion2
│           ├── name = DepA
│           ├── spans
│           │   └── 0
│           │       └── Span
│           │           ├── id = span3


## Storing `ProteinSequence`s in a PostgreSQL database



In [4]:
from sdrdm_database import DBConnector

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 
If docker is not installed on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Delete contianers

>```    
>docker rm -vf $(docker ps -aq)
>docker rmi -f $(docker images -aq)
>```

### Connect to the PostgreSQL database

In [ ]:
import toml

# Establish a connection to the database
db = DBConnector(**toml.load(open("./env.toml")))

### Create tables for `ProteinInfo`

In [ ]:
db.create_tables(
    model=ProteinInfo,
    markdown_path="/Users/max/Documents/GitHub/pyeed/specifications/data_model.md",
)

In [ ]:
# See all created table names
db.connection.list_tables()

### Populate the database with `ProteinSequence`s

In [ ]:
# Insert all blast results into the database
db.insert(*blast_results, verbose=True)

### Look at entries in the database

In [ ]:
db.connection.table("ProteinInfo_organism")

In [ ]:
# Lets filter the blast results for a specific organism
target = "Drosophila melanogaster"

# First, join the ProteinSequence table with the ProteinSequence_organism table
prot_seqs = db.connection.table("ProteinInfo")
organisms = db.connection.table("ProteinInfo_organism")
joined = prot_seqs.join(
    organisms,
    prot_seqs.ProteinInfo_id == organisms.ProteinInfo_id,
    rname="organism_{name}",
)

# Next, filter the joined table for the target organism
filtered = joined.filter(joined.organism_name == target)
filtered

# Finally, we can get the corresponding ProteinSequence objects
results = db.get("ProteinInfo", filtered)
print(len(results))